In [1]:
import os
import json
import minsearch # !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
from openai import OpenAI
client = OpenAI()

In [2]:
prompt_template = """
you are a course teaching assistant. 
Answer the question based on the context. 
use only the facts from the context when answering the question.
if the context does not contain the answer, output none

question: {question}
context: {context}
"""

In [3]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [5]:
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query = 'the course has already started, can i still enroll?',
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [6]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [7]:
q = "is it too late to join the course?"
prompt = prompt_template.format(question=q, context=context).strip()

In [8]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)
response.choices[0].message.content

"No, it is not too late to join the course. Even if you don't register initially, you are still eligible to submit the homeworks. However, be mindful of the deadlines for the final projects."